# CS 178 Project
## Import part

In [1]:
import os
import numpy as np
import pandas as pd
import cv2

from sklearn import svm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Read image for train

In [2]:
def read_pic(data, categories):
    folder_path = 'images/'
    image_paths = data.iloc[:, 1]
    labels = data.iloc[:, 2].str.lower()
    X = []
    label_to_int = {category: i for i, category in enumerate(categories)}
    integer_labels = labels.map(label_to_int)
    y = torch.LongTensor(integer_labels.to_numpy())
    for filename in image_paths:
        if filename.endswith(('.png', '.jpg', '.jpeg')):  # Check for common image file extensions
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (350, 350))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            if img is not None:
                X.append(torch.Tensor(img))
    X = torch.stack(X).unsqueeze(1) # convert to pytorch tensor (batch, channels, w, h)
    return X, y
    
categories1 = [
    "anger", 
    "surprise", 
    "disgust", 
    "fear", 
    "neutral", 
    "happiness", 
    "sadness",
    "contempt"
] # This is for Legend
categories2 = [
    "anger", 
    "surprise", 
    "disgust", 
    "fear", 
    "neutral", 
    "happiness", 
    "sad",
    "contempt"
] # This is for 500_picts

Xtr, ytr = read_pic(pd.read_csv('data/legend.csv'), categories1)
Xva, yva = read_pic(pd.read_csv('data/500_picts_satz.csv'), categories2)

print(Xtr.shape)
print(Xva.shape)
print(ytr)
print(yva)

tensor([0, 1, 2,  ..., 5, 5, 5])
tensor([5, 4, 0, 0, 0, 4, 0, 6, 4, 4, 0, 3, 3, 0, 4, 3, 4, 0, 3, 5, 0, 3, 6, 6,
        5, 6, 5, 0, 0, 6, 5, 5, 5, 0, 5, 6, 4, 5, 0, 0, 0, 0, 0, 5, 5, 5, 5, 5,
        4, 0, 5, 5, 3, 5, 5, 4, 0, 5, 4, 3, 5, 5, 3, 5, 5, 3, 5, 5, 0, 5, 0, 3,
        5, 0, 5, 5, 5, 5, 5, 0, 6, 4, 0, 4, 5, 4, 0, 3, 3, 5, 4, 0, 5, 0, 0, 5,
        4, 0, 4, 0, 0, 0, 4, 4, 5, 5, 4, 6, 5, 5, 5, 0, 3, 6, 4, 4, 0, 4, 5, 5,
        6, 0, 0, 0, 3, 3, 0, 5, 3, 0, 6, 0, 0, 0, 5, 0, 3, 5, 5, 4, 4, 4, 5, 0,
        0, 0, 0, 3, 0, 0, 3, 3, 3, 5, 5, 5, 5, 4, 5, 5, 5, 4, 5, 5, 4, 4, 3, 3,
        3, 3, 3, 4, 3, 4, 4, 3, 4, 0, 0, 4, 6, 4, 4, 5, 0, 5, 0, 0, 5, 5, 5, 5,
        5, 5, 5, 0, 4, 4, 5, 5, 6, 0, 4, 0, 0, 3, 3, 5, 3, 5, 5, 5, 5, 3, 0, 0,
        0, 0, 5, 5, 0, 5, 5, 0, 6, 5, 4, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        4, 4, 4, 4, 4, 4, 4, 4, 0, 6, 4, 4, 4, 4, 0, 0, 0, 5, 5, 5, 4, 0, 5, 6,
        4, 0, 0, 5, 5, 5, 0, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 0, 5, 5,
       

## CNN Test
### Define a Convolutional Neural Network
To calculate the layer after each convolutional layer, we should do:

In [3]:
class Net(nn.Module):
    def __init__(self, conv_layer_channels, fc_layer_channels, kernel_size = 5):
        super().__init__()

        # Pooling Layer
        self.pool = nn.MaxPool2d(2, 2)

        # Convolutional Layer
        self.conv1 = nn.Conv2d(1, 6, kernel_size) # 1 because its greyscale (in_channels, out_channels, kernel_size) (6, 350, 350) -> (6, 173, 173)
        self.conv2 = nn.Conv2d(6, 16, kernel_size) # (16, 84, 84)

        # Linear Layer
        self.fc1 = nn.Linear(16 * 84 * 84, 120) # shape of image after pooling twice (16, 84, 84)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 8) # 8 is label count

    def forward(self, x):
        # Conv1 -> Pool -> Conv2 -> Pool -> Lin1 -> Lin2 -> Lin3
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net([6, 16], [120, 84])

In [4]:
for i in range(len(ytr)):
    if ytr[i] < 0:
        print(i)

### Define a Loss function and optimizer

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

### Train the network

In [6]:
batch_size = 2
train_dataset = torch.utils.data.TensorDataset(Xtr, ytr)
test_dataset = torch.utils.data.TensorDataset(Xva, yva)
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()
        outputs = net(inputs) 
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,   100] loss: 0.201
[1,   200] loss: 0.082
[1,   300] loss: 0.075
[1,   400] loss: 0.064
[1,   500] loss: 0.059
[1,   600] loss: 0.055
[1,   700] loss: 0.059
[1,   800] loss: 0.057
[1,   900] loss: 0.052
[1,  1000] loss: 0.061
[1,  1100] loss: 0.057
[1,  1200] loss: 0.057
[1,  1300] loss: 0.057
[1,  1400] loss: 0.053
[1,  1500] loss: 0.050
[1,  1600] loss: 0.052
[1,  1700] loss: 0.054
[1,  1800] loss: 0.052
[1,  1900] loss: 0.050
[1,  2000] loss: 0.054
[1,  2100] loss: 0.055
[1,  2200] loss: 0.049
[1,  2300] loss: 0.053
[1,  2400] loss: 0.055
[1,  2500] loss: 0.055
[1,  2600] loss: 0.055
[1,  2700] loss: 0.053
[1,  2800] loss: 0.051
[1,  2900] loss: 0.053
[1,  3000] loss: 0.050
[1,  3100] loss: 0.047
[1,  3200] loss: 0.054
[1,  3300] loss: 0.056
[1,  3400] loss: 0.048
[1,  3500] loss: 0.054
[1,  3600] loss: 0.053
[1,  3700] loss: 0.050
[1,  3800] loss: 0.048
[1,  3900] loss: 0.054
[1,  4000] loss: 0.053
[1,  4100] loss: 0.048
[1,  4200] loss: 0.047
[1,  4300] loss: 0.056
[1,  4400] 

KeyboardInterrupt: 

In [ ]:
PATH = './model/test_net.pth'
torch.save(net.state_dict(), PATH)

### Test model

In [ ]:
net = Net([6, 16], [120, 84])
net.load_state_dict(torch.load(PATH))

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network: {100 * correct // total} %')